In [35]:
import json

def dive_into_category(objects, arr):
    for obj in objects:
        arr.append(obj)
        if obj["child_count"] > 0:
            dive_into_category(obj["children"], arr)
            
def build_category_arr():
    categories_tree = json.load(open('categories.json'))
    arr = []
    dive_into_category(categories_tree, arr)
    return arr

def get_subtree(categories, name_en):
    subtree = []
    for category_obj in categories:
        if category_obj["name_en"] != name_en:
            continue
        subtree.append(category_obj)
        break
    if len(subtree) == 0:
        return []
    while subtree[-1]["parent"] != None:
        parent_id = subtree[-1]["parent"]
        for category_obj in categories:
            if category_obj["id_catalog_category"] != parent_id:
                continue
            subtree.append(category_obj)
            break
    return subtree
        
def get_category_lvl(categories, name_en, level):
    subtree = get_subtree(categories, name_en)
    if len(subtree) == 0:
        return []
    if level >= len(subtree):
        return subtree[0]
    return subtree[-level]
    
categories_arr = build_category_arr()

In [36]:
import csv
import fastText

In [37]:
#modelFile = '/Users/dmitriy.zenin/dev/lab/wiki.en.bin'
modelFile = '/Users/dmitriy.zenin/dev/lab/data_cskip_formated_model/data_model.bin'

In [38]:
model = fastText.load_model(modelFile)

In [39]:
inFile    = '/Users/dmitriy.zenin/dev/lab/10000.csv'
outFile   = '/Users/dmitriy.zenin/dev/lab/10000_out.csv'

In [40]:
help(model)

Help on _FastText in module fastText.FastText object:

class _FastText(builtins.object)
 |  This class defines the API to inspect models and should not be used to
 |  create objects. It will be returned by functions such as load_model or
 |  train.
 |  
 |  In general this API assumes to be given only unicode for Python2 and the
 |  Python3 equvalent called str for any string-like arguments. All unicode
 |  strings are then encoded as UTF-8 and fed to the fastText C++ API.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_dimension(self)
 |      Get the dimension (size) of a lookup vector (hidden layer).
 |  
 |  get_input_matrix(self)
 |      Get a copy of the full input matrix of a Model. This only
 |      works if the model is not quantized.
 |  
 |  get_input_vector(self, ind)
 |      Given an index, get the corresponding vector of the Input Matrix.
 |  
 |  get_labels(self, include

In [60]:
import re
extra_symbols_pattern = re.compile(r"[\[\]_\"|\-;:.\!?,'/()@#$%^&\*\n]")
small_phrases_pattern = re.compile(r"\s\S{,2}\s")
extra_spaces_pattern  = re.compile(r'\s{2,}')

In [42]:
with open(outFile, "w", newline='') as wfile:
    wtr = csv.writer(wfile)
    with open(inFile, newline='') as rfile:
        rdr = csv.reader(rfile, delimiter=',', quotechar='"')
        i = 1
        for row in rdr:
            if len(row) < 2:
                continue
            categories = row[1].split('|')
            sentence = ' '.join(row[0].splitlines())
            sv = model.get_sentence_vector(sentence)
            sentence = re.sub(extra_symbols_pattern, "", sentence)
            sentence = re.sub(small_phrases_pattern, "", sentence)
            sentence = re.sub(extra_spaces_pattern, " ", sentence)
            sentence = sentence.lower()
            wtr.writerow((sentence, sv, '|'.join(sorted(categories, key=str.lower))))
            if i % 1000 == 0:
                print("done: ", i)
            i += 1

done:  1000
done:  2000
done:  3000
done:  4000
done:  5000
done:  6000


In [43]:
import numpy as np

naive_categories = dict()

def get_category(categories, max_color):
    target_category = "none"
    ncs = categories.split('|')
    for nc in ncs:
        c = get_category_lvl(categories_arr, nc, 1)
        if len(c) == 0:
            continue
        target_category = c["name_en"]
        break
    if target_category != "none" and target_category not in naive_categories:
        naive_categories[target_category] = max_color
        max_color += 1
    return (target_category, max_color)

X = list()
Y = list()
colors = list()

with open(outFile, newline='') as rfile:
    rdr = csv.reader(rfile, delimiter=',', quotechar='"')
    max_color = 1
    for row in rdr:
        v = np.fromstring(row[1][1:-1], dtype=np.float64, sep=' ')
        (category, max_color) = get_category(row[2], max_color)
        if category == "none":
            continue
        X.append(v)
        Y.append(category)
        colors.append(naive_categories[category])
    X = np.asarray(X)
    colors = np.asarray(colors)

In [44]:
print(X.shape, len(Y), len(colors), len(naive_categories))

(6765, 100) 6765 6765 23


In [45]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=3)
pca.fit(X)
pca_X = pca.transform(X)

In [46]:
import pandas as pd
df = pd.DataFrame(data=pca_X, 
             columns=['X', 'Y', 'Z'])
df['class'] = colors
df['text_class'] = Y
df.head()

,X,Y,Z,class,text_class
0,-0.109217,-0.015384,-0.101650,1,Health & Beauty
1,0.066160,-0.002414,0.003259,2,Fashion
2,0.045757,0.035837,0.102787,2,Fashion
3,0.198120,0.040379,-0.010219,3,Bags and Travel
4,-0.067842,0.055096,-0.019825,1,Health & Beauty


In [47]:
import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

plotly.offline.init_notebook_mode(connected=True)

traces = []
for label, data in df.groupby('text_class'):
    trace = go.Scatter3d(
        name=label,
        x=data['X'],
        y=data['Y'],
        z=data['Z'],
        mode='markers',
        marker=dict(
            size=5,
            line=dict(
                #color='rgba(217, 217, 217, 0.14)',
                width=0.1
            ),
            opacity=0.6
        )
    )
    traces.append(trace)

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
#fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='simple-3d-scatter')
plotly.offline.iplot({
    "data": traces,
    "layout": layout
})

In [61]:
inFileTrain  = '/Users/dmitriy.zenin/dev/lab/data.csv'
outFileTrain = '/Users/dmitriy.zenin/dev/lab/data_train.csv'

In [62]:
with open(outFileTrain, "w", newline='') as wfile:
    with open(inFileTrain, newline='') as rfile:
        rdr = csv.reader(rfile, delimiter=',', quotechar='"')
        i = 1
        for row in rdr:
            if len(row) < 2:
                continue
            title = re.sub(extra_symbols_pattern, "", row[0])
            title = re.sub(small_phrases_pattern, "", title)
            title = re.sub(extra_spaces_pattern, " ", title)
            title = title.lower()
            description = re.sub(extra_symbols_pattern, "", row[2])
            description = re.sub(small_phrases_pattern, "", description)
            description = re.sub(extra_spaces_pattern, " ", description)
            description = description.lower()
            wfile.write(title + " " + description)
            if i % 1000 == 0:
                print("done: ", i)
            i += 1

done:  1000
done:  2000
done:  3000
done:  4000
done:  5000
done:  6000
done:  7000
done:  8000
done:  9000
done:  10000
done:  11000
done:  12000
done:  13000
done:  14000
done:  15000
done:  16000
done:  17000
done:  18000
done:  19000
done:  20000
done:  21000
done:  22000
done:  23000
done:  24000
done:  25000
done:  26000
done:  27000
done:  28000
done:  29000
done:  30000
done:  31000
done:  32000
done:  33000
done:  34000
done:  35000
done:  36000
done:  37000
done:  38000
done:  39000
done:  40000
done:  41000
done:  42000
done:  43000
done:  44000
done:  45000
done:  46000
done:  47000
done:  48000
done:  49000
done:  50000
done:  51000
done:  52000
done:  53000
done:  54000
done:  55000
done:  56000
done:  57000
done:  58000
done:  59000
done:  60000
done:  61000
done:  62000
done:  63000
done:  64000
done:  65000
done:  66000
done:  67000
done:  68000
done:  69000
done:  70000
done:  71000
done:  72000
done:  73000
done:  74000
done:  75000
done:  76000
done:  77000
done:  7

done:  594000
done:  595000
done:  596000
done:  597000
done:  598000
done:  599000
done:  600000
done:  601000
done:  602000
done:  603000
done:  604000
done:  605000
done:  606000
done:  607000
done:  608000
done:  609000
done:  610000
done:  611000
done:  612000
done:  613000
done:  614000
done:  615000
done:  616000
done:  617000
done:  618000
done:  619000
done:  620000
done:  621000
done:  622000
done:  623000
done:  624000
done:  625000
done:  626000
done:  627000
done:  628000
done:  629000
done:  630000
done:  631000
done:  632000
done:  633000
done:  634000
done:  635000
done:  636000
done:  637000
done:  638000
done:  639000
done:  640000
done:  641000
done:  642000
done:  643000
done:  644000
done:  645000
done:  646000
done:  647000
done:  648000
done:  649000
done:  650000
done:  651000
done:  652000
done:  653000
done:  654000
done:  655000
done:  656000
done:  657000
done:  658000
done:  659000
done:  660000
done:  661000
done:  662000
done:  663000
done:  664000
done: 

done:  1169000
done:  1170000
done:  1171000
done:  1172000
done:  1173000
done:  1174000
done:  1175000
done:  1176000
done:  1177000
done:  1178000
done:  1179000
done:  1180000
done:  1181000
done:  1182000
done:  1183000
done:  1184000
done:  1185000
done:  1186000
done:  1187000
done:  1188000
done:  1189000
done:  1190000
done:  1191000
done:  1192000
done:  1193000
done:  1194000
done:  1195000
done:  1196000
done:  1197000
done:  1198000
done:  1199000
done:  1200000
done:  1201000
done:  1202000
done:  1203000
done:  1204000
done:  1205000
done:  1206000
done:  1207000
done:  1208000
done:  1209000
done:  1210000
done:  1211000
done:  1212000
done:  1213000
done:  1214000
done:  1215000
done:  1216000
done:  1217000
done:  1218000
done:  1219000
done:  1220000
done:  1221000
done:  1222000
done:  1223000
done:  1224000
done:  1225000
done:  1226000
done:  1227000
done:  1228000
done:  1229000
done:  1230000
done:  1231000
done:  1232000
done:  1233000
done:  1234000
done:  123

done:  1716000
done:  1717000
done:  1718000
done:  1719000
done:  1720000
done:  1721000
done:  1722000
done:  1723000
done:  1724000
done:  1725000
done:  1726000
done:  1727000
done:  1728000
done:  1729000
done:  1730000
done:  1731000
done:  1732000
done:  1733000
done:  1734000
done:  1735000
done:  1736000
done:  1737000
done:  1738000
done:  1739000
done:  1740000
done:  1741000
done:  1742000
done:  1743000
done:  1744000
done:  1745000
done:  1746000
done:  1747000
done:  1748000
done:  1749000
done:  1750000
done:  1751000
done:  1752000
done:  1753000
done:  1754000
done:  1755000
done:  1756000
done:  1757000
done:  1758000
done:  1759000
done:  1760000
done:  1761000
done:  1762000
done:  1763000
done:  1764000
done:  1765000
done:  1766000
done:  1767000
done:  1768000
done:  1769000
done:  1770000
done:  1771000
done:  1772000
done:  1773000
done:  1774000
done:  1775000
done:  1776000
done:  1777000
done:  1778000
done:  1779000
done:  1780000
done:  1781000
done:  178

done:  2265000
done:  2266000
done:  2267000
done:  2268000
done:  2269000
done:  2270000
done:  2271000
done:  2272000
done:  2273000
done:  2274000
done:  2275000
done:  2276000
done:  2277000
done:  2278000
done:  2279000
done:  2280000
done:  2281000
done:  2282000
done:  2283000
done:  2284000
done:  2285000
done:  2286000
done:  2287000
done:  2288000
done:  2289000
done:  2290000
done:  2291000
done:  2292000
done:  2293000
done:  2294000
done:  2295000
done:  2296000
done:  2297000
done:  2298000
done:  2299000
done:  2300000
done:  2301000
done:  2302000
done:  2303000
done:  2304000
done:  2305000
done:  2306000
done:  2307000
done:  2308000
done:  2309000
done:  2310000
done:  2311000
done:  2312000
done:  2313000
done:  2314000
done:  2315000
done:  2316000
done:  2317000
done:  2318000
done:  2319000
done:  2320000
done:  2321000
done:  2322000
done:  2323000
done:  2324000
done:  2325000
done:  2326000
done:  2327000
done:  2328000
done:  2329000
done:  2330000
done:  233

...

done:  28960000
done:  28961000
done:  28962000
done:  28963000
done:  28964000
done:  28965000
done:  28966000
done:  28967000
done:  28968000
done:  28969000
done:  28970000
done:  28971000
done:  28972000
done:  28973000
done:  28974000
done:  28975000
done:  28976000
done:  28977000
done:  28978000
done:  28979000
done:  28980000
done:  28981000
done:  28982000
done:  28983000
done:  28984000
done:  28985000
done:  28986000
done:  28987000
done:  28988000
done:  28989000
done:  28990000
done:  28991000
done:  28992000
done:  28993000
done:  28994000
done:  28995000
done:  28996000
done:  28997000
done:  28998000
done:  28999000
done:  29000000
done:  29001000
done:  29002000
done:  29003000
done:  29004000
done:  29005000
done:  29006000
done:  29007000
done:  29008000
done:  29009000
done:  29010000
done:  29011000
done:  29012000
done:  29013000
done:  29014000
done:  29015000
done:  29016000
done:  29017000
done:  29018000
done:  29019000
done:  29020000
done:  29021000
done:  2

done:  29475000
done:  29476000
done:  29477000
done:  29478000
done:  29479000
done:  29480000
done:  29481000
done:  29482000
done:  29483000
done:  29484000
done:  29485000
done:  29486000
done:  29487000
done:  29488000
done:  29489000
done:  29490000
done:  29491000
done:  29492000
done:  29493000
done:  29494000
done:  29495000
done:  29496000
done:  29497000
done:  29498000
done:  29499000
done:  29500000
done:  29501000
done:  29502000
done:  29503000
done:  29504000
done:  29505000
done:  29506000
done:  29507000
done:  29508000
done:  29509000
done:  29510000
done:  29511000
done:  29512000
done:  29513000
done:  29514000
done:  29515000
done:  29516000
done:  29517000
done:  29518000
done:  29519000
done:  29520000
done:  29521000
done:  29522000
done:  29523000
done:  29524000
done:  29525000
done:  29526000
done:  29527000
done:  29528000
done:  29529000
done:  29530000
done:  29531000
done:  29532000
done:  29533000
done:  29534000
done:  29535000
done:  29536000
done:  2

done:  29988000
done:  29989000
done:  29990000
done:  29991000
done:  29992000
done:  29993000
done:  29994000
done:  29995000
done:  29996000
done:  29997000
done:  29998000
done:  29999000
done:  30000000
done:  30001000
done:  30002000
done:  30003000
done:  30004000
done:  30005000
done:  30006000
done:  30007000
done:  30008000
done:  30009000
done:  30010000
done:  30011000
done:  30012000
done:  30013000
done:  30014000
done:  30015000
done:  30016000
done:  30017000
done:  30018000
done:  30019000
done:  30020000
done:  30021000
done:  30022000
done:  30023000
done:  30024000
done:  30025000
done:  30026000
done:  30027000
done:  30028000
done:  30029000
done:  30030000
done:  30031000
done:  30032000
done:  30033000
done:  30034000
done:  30035000
done:  30036000
done:  30037000
done:  30038000
done:  30039000
done:  30040000
done:  30041000
done:  30042000
done:  30043000
done:  30044000
done:  30045000
done:  30046000
done:  30047000
done:  30048000
done:  30049000
done:  3

done:  30501000
done:  30502000
done:  30503000
done:  30504000
done:  30505000
done:  30506000
done:  30507000
done:  30508000
done:  30509000
done:  30510000
done:  30511000
done:  30512000
done:  30513000
done:  30514000
done:  30515000
done:  30516000
done:  30517000
done:  30518000
done:  30519000
done:  30520000
done:  30521000
done:  30522000
done:  30523000
done:  30524000
done:  30525000
done:  30526000
done:  30527000
done:  30528000
done:  30529000
done:  30530000
done:  30531000
done:  30532000
done:  30533000
done:  30534000
done:  30535000
done:  30536000
done:  30537000
done:  30538000
done:  30539000
done:  30540000
done:  30541000
done:  30542000
done:  30543000
done:  30544000
done:  30545000
done:  30546000
done:  30547000
done:  30548000
done:  30549000
done:  30550000
done:  30551000
done:  30552000
done:  30553000
done:  30554000
done:  30555000
done:  30556000
done:  30557000
done:  30558000
done:  30559000
done:  30560000
done:  30561000
done:  30562000
done:  3

done:  31014000
done:  31015000
done:  31016000
done:  31017000
done:  31018000
done:  31019000
done:  31020000
done:  31021000
done:  31022000
done:  31023000
done:  31024000
done:  31025000
done:  31026000
done:  31027000
done:  31028000
done:  31029000
done:  31030000
done:  31031000
done:  31032000
done:  31033000
done:  31034000
done:  31035000
done:  31036000
done:  31037000
done:  31038000
done:  31039000
done:  31040000
done:  31041000
done:  31042000
done:  31043000
done:  31044000
done:  31045000
done:  31046000
done:  31047000
done:  31048000
done:  31049000
done:  31050000
done:  31051000
done:  31052000
done:  31053000
done:  31054000
done:  31055000
done:  31056000
done:  31057000
done:  31058000
done:  31059000
done:  31060000
done:  31061000
done:  31062000
done:  31063000
done:  31064000
done:  31065000
done:  31066000
done:  31067000
